<a href="https://colab.research.google.com/github/RaianaSeixas/Paper-1-Final-Results/blob/master/Stats_2000_Shubert3_ALL3_D_10_0_5_0_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyade-python

In [2]:
!pip install PyFDE

     |████████████████████████████████| 399kB 2.8MB/s 
  Created wheel for PyFDE: filename=PyFDE-1.3.0-cp36-cp36m-linux_x86_64.whl size=969266 sha256=beda492e6dc0b845d191fddd9f10b4f0a19eaac92bb7eb93922dab58da62b5f8
  Stored in directory: /root/.cache/pip/wheels/7d/42/ba/abf21db37934e576c74fc2c9504b5cd97eb6a99e3dec1743c0
Successfully built PyFDE


In [3]:
import pyade.ilshade 
import pyade.sade 
import pyade.jso
import pyade.shade
import pyade.jade 
import pyfde
import numpy as np
import pandas as pd
import time

In [4]:
!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 132 (delta 3), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (132/132), 200.33 KiB | 5.56 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Raianars_July_2020


In [5]:
import Go2Ann
import de_soma_Insight
import de_bounds_matriz
from AvaliaX import AvaliaX
from GeraIndices import GeraIndices

In [6]:
import Function
import Go2MinMax

In [7]:
ITE=2000 #10 # PONTOS quantas vezes vai alteral NPAR 
NPAR=100 # NUMBER OF PARTICLES
dim=10
RUNS=50
fchoice='Shubert3'  #US
Fun=Function.Shubert3

MIN,MAX=Go2MinMax.Intervalo(dim,fchoice)
bounds = [(0,0)]*dim

for i in range(dim):
  bounds[i] = (MIN[i],MAX[i])

start_time = time.time() 

In [8]:
############################### INPUT INFORMATION (FRANNK)
Fc=0.1 # fator de correcao parametros da ANN
its=1 #10 # quantas vezes vai usar o NPAR antes de alterar de novo
mut=0.25
crossp=0.25
Fi=mut;CRi=crossp;# para impressão
PAR=len(MAX) #NUM DE PARAMETROS A SER OTIMIZADOS

In [9]:
# JADE
algorithm = pyade.jso
BEST_AGENTS=[]
for i in range(RUNS):
  ANSWER_AGENTS=[]
  solver = pyfde.JADE(Fun, n_dim=dim, n_pop=NPAR, limits=bounds, minimize=True)
  solver.p = 0.05
  solver.c = 0.1
  solver.cr = 0.5
  solver.f = 0.5
  best, fit = solver.run(n_it=ITE)
  for x in best:
    ANSWER_AGENTS.append(x)
  ANSWER_AGENTS.append(fit)
  BEST_AGENTS.append(ANSWER_AGENTS)
BEST_AGENTS=np.array(BEST_AGENTS)
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()
nome='JADE_Fo-'+str(0)+'_CRo-'+str(0)
print(nome,'NPAR',NPAR,'Iterações=',ITE,'Dim=',dim ,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print( '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

JADE_Fo-0_CRo-0 NPAR 100 Iterações= 2000 Dim= 10 Min/Max= -10 / 10 _ Shubert3


MÉDIA:  148.379500257106
MIN __: 148.379500257106
MAX__:  148.37950025710603
STD __: 1.8852829216231156e-14
Tempo Médio= 17.432 segundos ---
50 Rodadas

BEST:  [ -1.11409969  -1.11409969  -1.11409969  -7.39728499  -7.39728499
  -1.11409968   5.16908561  -7.39728499  -7.39728499  -7.39728499
 148.37950026]


In [10]:
# FRANNK START HERE !!!!!

In [11]:
!git clone https://github.com/ucfilho/ANN_read_matrix #clonar do Github
%cd ANN_read_matrix

Cloning into 'ANN_read_matrix'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/Raianars_July_2020/ANN_read_matrix


In [12]:
ANN_F=pd.read_csv('ANN_INSIGHT_Valor_F.csv')
ANN_CR=pd.read_csv('ANN_INSIGHT_CR.csv')
syn0_F,syn1_F,X_max_F,X_min_F=Go2Ann.GetMatriz(ANN_F)
syn0_CR,syn1_CR,X_max_CR,X_min_CR=Go2Ann.GetMatriz(ANN_CR)
fields = ['DI RELATIVO','FRAC Its','Fo','CRo']

#TOTAL=7*dim*its*ITE
TOTAL=10*dim*its*ITE
TOTAL=int(TOTAL/2)   # CONFERIR SE ESTE EH O MELHOR LUGAR

dimensions=len(MAX)
start_time = time.time() #INICIALIZA CONTAGEM DO TEMPO

MAX_P=np.zeros(ITE) # ATENCAO para variar F e Cr MAX_P passa a ser 3 vezes maior
MIN_P=np.zeros(ITE) # ATENCAO para variar F e Cr MIN_P passa a ser 3 vezes maior

for i in range(dim):
  MAX_P[i]=MAX[i] 
  MIN_P[i]=MIN[i]

rodadas=1
for i in range(rodadas):
  SOMA=0
  pop = np.random.rand(NPAR, dimensions)
  diff=np.zeros(PAR)
  for ii in range(PAR):
    diff[ii] = MAX[ii]-MIN[ii]
  X = MIN + pop * diff
    
  PONTOS=ITE # DIVISOES DA POPULACAO
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]
for i in range(RUNS):
    MAT=[] 
    mut=Fi
    crossp=CRi
    SOMA=0
    SOMAf=0
    start_ite = time.time()
    delta_time=(start_ite-start_time)/60
    print('ite=',i,'time=',delta_time)
    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(PAR)
    for ii in range(PAR):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff
    Xf = MIN + pop * diff
    
    DIo=1e99
  
    CRo=crossp;Fo=mut;
    Fitness = np.asarray([Fun(ind) for ind in X])

    XY,BEST_XY,BESTo,FOBESTo=AvaliaX(X,Fitness)
    nrow,ncol=X.shape

    QUANT=17;MAT_INDo=np.zeros((1,QUANT)); # quantos indices esta fazendo
    MAT_INDo[0,5]=nrow  # VALOR NP
    MAT_INDo[0,6]=Fo    # VALOR Fo   
    MAT_INDo[0,7]=CRo   # VALOR CRo
  
    MAT_INDo=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                        syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                        X_min_CR,Fc,fields,Fun)
  
    DIo=np.copy(MAT_INDo[0,0])
    MAT_INDo[0,15]=Fo    # VALOR Fo   
    MAT_INDo[0,16]=CRo   # VALOR CRo
    #ind_crossp=ITE-1

    
    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,NPAR,its,Fun,X,SOMA,TOTAL)
      #Xf,BESTf,FOBESTf,XYf,BEST_XYf,SOMAf= de_soma_Insight.de(MAX,MIN, Fi, CRi, NPAR,its,Fun,Xf,SOMAf,TOTAL)

      
      MAT_IND=GeraIndices(X,BESTo,FOBESTo,DIo,MAT_INDo,SOMA,TOTAL,syn0_F,
                          syn1_F,X_max_F,X_min_F,syn0_CR,syn1_CR,X_max_CR,
                          X_min_CR,Fc,fields,Fun)
    
      BESTo=np.copy(BEST);FOBESTo=np.copy(FOBEST);
      MAT.append(MAT_IND.ravel())
    
      MAT_INDo[0,6]=MAT_IND[0,15]   # VALOR Fo   
      MAT_INDo[0,7]=MAT_IND[0,16]   # VALOR CRo
      mut=MAT_IND[0,15]
      crossp=MAT_IND[0,16]

      Fitness = np.asarray([Fun(ind) for ind in X])
      #Fitnessf = np.asarray([Fun(ind) for ind in Xf])
      Maior=np.argmax(Fitness)
      #Maiorf=np.argmax(Fitnessf)
      Menor=np.argmin(Fitness)
      #Menorf=np.argmin(Fitnessf)

      #X[Maior,:]=np.copy(Xf[Menorf,:])
      #Xf[Maiorf,:]=np.copy(X[Menor,:])
      
    BEST_AGENTS.append(BEST_XY)
    MAT_stat.append(MAT)
    #print("GlobalParam e Solucao:", BEST,"Funcao=",FOBEST)
    
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

print()
nome='_ANN_Div3x4_Fo-'+str(Fi)+'_CRo-'+str(CRi)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_POP_VAR_F_CR','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', PAR,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

ite= 0 time= 9.032090504964193e-06
ite= 1 time= 2.9368321816126506
ite= 2 time= 5.889270416895548
ite= 3 time= 8.822773849964141
ite= 4 time= 11.772661431630452
ite= 5 time= 14.714726356665293
ite= 6 time= 17.647135285536447
ite= 7 time= 20.58095959822337
ite= 8 time= 23.519516650835673
ite= 9 time= 26.46641302506129
ite= 10 time= 29.412597974141438
ite= 11 time= 32.362558821837105
ite= 12 time= 35.30072705745697
ite= 13 time= 38.235675700505574
ite= 14 time= 41.181831097602846
ite= 15 time= 44.130749094486234
ite= 16 time= 47.080965133508045
ite= 17 time= 50.01336763302485
ite= 18 time= 52.957943165302275
ite= 19 time= 55.893783660729724
ite= 20 time= 58.82528516848882
ite= 21 time= 61.75974161624909
ite= 22 time= 64.69777203798294
ite= 23 time= 67.63054939905803
ite= 24 time= 70.57154391209285
ite= 25 time= 73.50842708746592
ite= 26 time= 76.42797483205796
ite= 27 time= 79.36212656100591
ite= 28 time= 82.29826744794846
ite= 29 time= 85.24573602279027
ite= 30 time= 88.17622691790262
i

In [13]:
#DE_ tradicional

!git clone https://github.com/ucfilho/Raianars_July_2020 #clonar do Github
%cd Raianars_July_2020

mut=0.5
crossp=0.5

SOMA=0
start_time = time.time()
BEST_AGENTS=[]
MAT_stat=[]

for i in range(RUNS):

    pop = np.random.rand(NPAR, dim)
    diff=np.zeros(dim)
    for ii in range(dim):
      diff[ii] = MAX[ii]-MIN[ii]
    X = MIN + pop * diff

    SOMA=0

    for k in range(ITE):
      X,BEST,FOBEST,XY,BEST_XY,SOMA= de_soma_Insight.de(MAX,MIN, mut, crossp,dim,its,Fun,X,SOMA,TOTAL)

    start_run = (time.time()-start_time)/60
    print('run=',i,'total=50','Fobj=',FOBEST,'Time=',start_run)  
    BEST_AGENTS.append(BEST_XY)

Cloning into 'Raianars_July_2020'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 132 (delta 3), reused 0 (delta 0), pack-reused 123
Receiving objects: 100% (132/132), 200.33 KiB | 6.91 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Raianars_July_2020/ANN_read_matrix/Raianars_July_2020
run= 0 total=50 Fobj= -148.37950025710603 Time= 0.22914928992589315
run= 1 total=50 Fobj= -148.36886514313113 Time= 0.4583802262941996
run= 2 total=50 Fobj= -148.37940160077906 Time= 0.6862888773282368
run= 3 total=50 Fobj= -148.30040258330916 Time= 0.913977034886678
run= 4 total=50 Fobj= -148.06423846744912 Time= 1.1420825163523356
run= 5 total=50 Fobj= -148.37950025630428 Time= 1.3703468958536784
run= 6 total=50 Fobj= -148.37950025707295 Time= 1.5980837186177572
run= 7 total=50 Fobj= -148.37904067372455 Time= 1.8264561931292216
run= 8 total=50 Fobj= -148.37949841131638 Time= 2.05589017868042
r

In [14]:
############################## Cálculo dos Parâmetros de Avaliação   
TIME = (time.time() - start_time)/RUNS
BEST_AGENTS=np.array(BEST_AGENTS)
BEST_AGENTS_ST= BEST_AGENTS[BEST_AGENTS[:,-1].argsort()]
BEST=BEST_AGENTS_ST[0]
WORSE=BEST_AGENTS_ST[-1]
MEAN=np.mean(BEST_AGENTS,axis=0)
MAXFO=BEST_AGENTS[:,-1].max()
MINFO=BEST_AGENTS[:,-1].min()
STD=BEST_AGENTS[:,-1].std()

nome='DE_STAT_Fo-'+str(mut)+'_CRo-'+str(crossp)+'_D-'+str(dim)+'_Interv-'+str(ITE)
print('DE_','TOTAL=',TOTAL,'Iterações=',ITE*its,'Dim=', dim,"Min/Max=",MIN[0],"/",MAX[0],'_',Fun.__name__)
print(nome, '\n')
#print('BEST: ',BEST)
#print('MÉDIA: ',MEAN)
print('MÉDIA: ',MEAN[-1])
print('MIN __:',MINFO)
print('MAX__: ',MAXFO)
print('STD __:',STD)
print("Tempo Médio= %.3f segundos ---" % (TIME))
print(RUNS,'Rodadas') 
print()
print('BEST: ',BEST)

DE_ TOTAL= 100000 Iterações= 2000 Dim= 10 Min/Max= -10 / 10 _ Shubert3
DE_STAT_Fo-0.5_CRo-0.5_D-10_Interv-2000 

MÉDIA:  -148.03211964439916
MIN __: -148.37950025710603
MAX__:  -144.83983732936323
STD __: 0.7617516034659659
Tempo Médio= 13.730 segundos ---
50 Rodadas

BEST:  [  -7.39728499    5.16908562    5.16908562   -7.397285     -1.11409969
   -1.11409969   -1.11409969   -1.11409969   -7.39728499   -7.39728499
 -148.37950026]
